In [1]:
from __future__ import absolute_import, division, print_function

import tensorflow as tf
print(tf.__version__)

tf.keras.backend.clear_session()  # For easy reset of notebook state.

2.0.0-alpha0


In [3]:
import pandas as pd
import numpy as np
import xarray as xr

In [21]:
import os

df = {}
file_path = os.path.join(os.getcwd(), "forex-data")
for f in os.listdir(file_path):
    if f.startswith('USD'):
        df[f] = pd.read_csv(os.path.join(file_path, f))
        df[f]['openTime'] = pd.to_datetime(df[f].Date, unit='ns')
        df[f]['openTime'] = pd.DatetimeIndex(df[f]['openTime'] ).round('S')
        df[f].index = df[f]['openTime']
        df[f].drop(columns=['openTime', 'Date'])
        df[f] = xr.DataArray(df[f], dims=['openTime', 'feature']) 
        
chart = xr.Dataset(df).to_array(dim='asset')
chart = chart.transpose("feature", "asset", "openTime")

print(chart.shape)
print(chart)

(7, 4, 74551)
<xarray.DataArray (feature: 7, asset: 4, openTime: 74551)>
array([[[1.16106, 1.16096, ..., 1.3446200000000001, 1.34423],
        [0.99372, 0.99384, ..., 1.01876, 1.01893],
        [6.15319, 6.15458, ..., 7.0689, 7.06722],
        [119.765, 119.796, ..., 116.979, 117.01700000000001]],

       [['2015-01-01T22:00:00.000000000Z', '2015-01-01T22:10:00.000000000Z',
         ..., '2016-12-30T21:40:00.000000000Z',
         '2016-12-30T21:50:00.000000000Z'],
        ['2015-01-01T22:00:00.000000000Z', '2015-01-01T22:10:00.000000000Z',
         ..., '2016-12-30T21:40:00.000000000Z',
         '2016-12-30T21:50:00.000000000Z'],
        ['2015-01-01T22:00:00.000000000Z', '2015-01-01T22:10:00.000000000Z',
         ..., '2016-12-30T21:40:00.000000000Z',
         '2016-12-30T21:50:00.000000000Z'],
        ['2015-01-01T22:00:00.000000000Z', '2015-01-01T22:10:00.000000000Z',
         ..., '2016-12-30T21:40:00.000000000Z',
         '2016-12-30T21:50:00.000000000Z']],

       ...,

       [[

In [15]:
print(chart.shape)
print(chart)

(4, 74551, 7)
<xarray.DataArray (asset: 4, openTime: 74551, feature: 7)>
array([[[1.16106, '2015-01-01T22:00:00.000000000Z', ..., 33,
         Timestamp('2015-01-01 22:00:00')],
        [1.16096, '2015-01-01T22:10:00.000000000Z', ..., 160,
         Timestamp('2015-01-01 22:10:00')],
        ...,
        [1.3446200000000001, '2016-12-30T21:40:00.000000000Z', ..., 257,
         Timestamp('2016-12-30 21:40:00')],
        [1.34423, '2016-12-30T21:50:00.000000000Z', ..., 896,
         Timestamp('2016-12-30 21:50:00')]],

       [[0.99372, '2015-01-01T22:00:00.000000000Z', ..., 23,
         Timestamp('2015-01-01 22:00:00')],
        [0.99384, '2015-01-01T22:10:00.000000000Z', ..., 25,
         Timestamp('2015-01-01 22:10:00')],
        ...,
        [1.01876, '2016-12-30T21:40:00.000000000Z', ..., 131,
         Timestamp('2016-12-30 21:40:00')],
        [1.01893, '2016-12-30T21:50:00.000000000Z', ..., 525,
         Timestamp('2016-12-30 21:50:00')]],

       [[6.15319, '2015-01-01T22:00:00.00

In [36]:
from pgportfolio.learn.nnagent import NNAgent
from pgportfolio.tools.configprocess import load_config

assets = ["CHF", "JPY", "CAD", "DKK"]
datetime = pd.date_range(start="01/01/2015", end="30/12/2016", freq="10min")

ModuleNotFoundError: No module named 'tensorflow.contrib'

In [35]:
from __future__ import absolute_import, print_function, division
from mercurius.data.candlereader import CandleReader
from pgportfolio.learn.nnagent import NNAgent
from pgportfolio.tools.configprocess import load_config
from pgportfolio.marketdata.globaldatamatrix import get_sentiment_data
from datetime import datetime, timedelta
import numpy as np
import xarray
import pandas as pd

def assets2symbols(assets):
    return [asset+"/BTC" for asset in assets]

def backtest(start, end, agent_index = 1,location = "train_package/"):
    """
    :param location: location of given model
    :param start: start date of backtest
    :param end: end date of backtest
    :param agent_index: index of agent
    :return: dictionary {"portfolio_changes_history":series, "portfolio_weights_history":dataframe,
                        "global_data":xarray}
    """
    config = load_config(agent_index)
    assets = ['ETH', 'LTC', 'XRP', 'ETC', 'DASH', 'XMR', 'XEM', 'FCT', 'GNT', 'ZEC']
    agent = NNAgent(config, location+str(agent_index)+"/netfile")
    reader = CandleReader(symbols=assets2symbols(assets), start=start, end=end, timeframe="30m")
    global_data = reader.get_data()
    global_sentiment = get_sentiment_data(assets, start, end, "30min")
    global_sentiment.columns = assets2symbols(global_sentiment.columns)
    global_sentiment = xarray.DataArray(global_sentiment.T.values[None, :, :], dims=["feature", "asset", "openTime"],
                     coords={"feature": ["sentiment"], "asset": global_sentiment.columns,
                             "openTime": global_sentiment.index})
    global_data = xarray.concat([global_data[:-1,:,:], global_sentiment], dim="feature")
    commission_rate = config["trading"]["trading_consumption"]

    window_size = config["input"]["window_size"]
    global_steps = len(global_data["openTime"]) - window_size
    last_w = np.zeros(len(assets)+1)
    last_w[0] = 1
    portfolio_changes_history = {}
    portfolio_weights_history = {}
    print(global_data)
    for i in range(global_steps):
        input_window = global_data[:,:,i:i+window_size]
        y = global_data[0, :, i+1]/global_data[0, :, i]
        datetime = input_window["openTime"][-1].values
        weights = agent.decide_by_history(input_window.values, last_w)
        transaction_cost = np.sum(np.abs(weights-last_w))*commission_rate
        last_w = weights
        portfolio_change = np.dot(y, weights[1:])*(1-transaction_cost)
        portfolio_changes_history[datetime] = portfolio_change
        portfolio_weights_history[datetime] = weights
    return {"portfolio_changes_history": pd.Series(portfolio_changes_history),
            "portfolio_weights_history": pd.DataFrame(portfolio_weights_history),
            "global_data": global_data}

TypeError: invalid indexer array, does not have integer dtype: array(None, dtype=object)